## Add diacritics

Add the root folder to the path, to use the files inside the `scripts` folder.

In [1]:
import sys
sys.path.append('..')

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Load the Gemini model

In [3]:
from google import genai

client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

In [4]:
from pprint import pprint
for model in client.models.list():
    pprint({
        "name": model.name,
        "input_token_limit": model.input_token_limit,
        "output_token_limit": model.output_token_limit,
        "supported_actions": model.supported_actions
    })
    print()

{'input_token_limit': 4096,
 'name': 'models/chat-bison-001',
 'output_token_limit': 1024,
 'supported_actions': ['generateMessage', 'countMessageTokens']}

{'input_token_limit': 8196,
 'name': 'models/text-bison-001',
 'output_token_limit': 1024,
 'supported_actions': ['generateText',
                       'countTextTokens',
                       'createTunedTextModel']}

{'input_token_limit': 1024,
 'name': 'models/embedding-gecko-001',
 'output_token_limit': 1,
 'supported_actions': ['embedText', 'countTextTokens']}

{'input_token_limit': 12288,
 'name': 'models/gemini-1.0-pro-vision-latest',
 'output_token_limit': 4096,
 'supported_actions': ['generateContent', 'countTokens']}

{'input_token_limit': 12288,
 'name': 'models/gemini-pro-vision',
 'output_token_limit': 4096,
 'supported_actions': ['generateContent', 'countTokens']}

{'input_token_limit': 2000000,
 'name': 'models/gemini-1.5-pro-latest',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countToke

### Create the prompts

First, set the system prompt.

In [20]:
system_prompt = """You are an expert in the Arabic language with a strong command of diacritics. Your task is to accurately add the appropriate diacritics to undiacritized Arabic text provided by the user, ensuring that each word is correctly vocalized based on its context.

For every word and character, apply the correct diacritics with precision. For example, the word اسمع should be transformed to اِسْمَعْ, not اسمعْ. Consider the meaning and structure of each sentence carefully to ensure accuracy.

Here is an example to illustrate the expected outcome:

Before diacritics:
اسمع أيها الإنسان وشاهد وتمعن أيها الموصوف بالعقل والحكمة. الحياة معجزة ظهرت منذ أربعة مليارات من الأعوام، ونحن بنو الإنسان لم يمض على وجودنا سوى مئتي ألف من الأعوام، إنها حقبة قصيرة لكنها كانت كافية لاختلال توازن أرضنا حتى باتت حياتنا على كوكبنا الأزرق مهددة بالزوال. استمعوا لهذه الحكاية، حكايتنا، حكاية الأرض.

After diacritics:
اِسْمَعْ أَيُّهَا الإِنْسَانُ وَشَاهِدْ وَتَمَعَّنْ أَيُّهَا المَوْصُوفُ بِالْعَقْلِ وَالْحِكْمَةِ. الْحَيَاةُ مُعْجِزَةٌ ظَهَرَتْ مُنْذُ أَرْبَعَةِ مِلْيَارَاتٍ مِنَ الْأَعْوَامِ، وَنَحْنُ بَنُو الْإِنْسَانِ لَمْ يَمْضِ عَلَى وُجُودِنَا سِوَى مِئَتَيْ أَلْفٍ مِنَ الْأَعْوَامِ، إِنَّهَا حِقْبَةٌ قَصِيرَةٌ لَكِنَّهَا كَانَتْ كَافِيَةً لِاخْتِلَالِ تَوَازُنِ أَرْضِنَا حَتَّى بَاتَتْ حَيَاتُنَا عَلَى كَوْكَبِنَا الْأَزْرَقِ مُهَدَّدَةً بِالزَّوَالِ. اِسْتَمِعُوا لِهَذِهِ الْحِكَايَةِ، حِكَايَتُنَا، حِكَايَةُ الْأَرْضِ.

Respond with the fully diacritized version of the text, ensuring that it is complete and accurate.
"""

Now, create the user query.

In [6]:
def get_user_query(undiacritized_text: str) -> str:
    return f"""I have some undiacritized Arabic text that needs diacritics. Please handle this task carefully, as diacritics significantly affect pronunciation and meaning. You have the full context, so do your best to add the appropriate diacritics. Kindly respond directly with the fully diacritized Arabic text. Thanks a lot! Here is the undiacritized Arabic text:

{undiacritized_text}
"""

### Get the undiacritized text

Step 1 : Extract the captions from the YouTube video

In [ ]:
from scripts.raw_text_extracter import YouTubeArabicCaptionExtractor

video_url = "https://www.youtube.com/watch?v=E9-k7wtS3bg"
caption_extractor = YouTubeArabicCaptionExtractor(video_url=video_url)
captions_text = caption_extractor.extract_captions()

In [10]:
len(captions_text)

70780

Step 2 : Process the raw captions

In [8]:
from scripts.caption_preprocessing import CaptionPreProcessor

caption_preprocessor = CaptionPreProcessor(raw_captions=captions_text)
undiacritized_text = caption_preprocessor.process_captions()

In [9]:
len(undiacritized_text)

23717

### Add the diacritics

In [21]:
from google.genai.types import GenerateContentConfig

response = client.models.generate_content(
    model='gemini-2.0-pro-exp-02-05',
    contents=get_user_query(undiacritized_text),
    config=GenerateContentConfig(
        system_instruction=[
            system_prompt
        ]
    )
)

print(response.text)

اِسْمَعْ أَيُّهَا الإِنْسَانُ وَشَاهِدْ وَتَمَعَّنْ أَيُّهَا المَوْصُوفُ بِالْعَقْلِ وَالْحِكْمَةِ. الْحَيَاةُ مُعْجِزَةٌ ظَهَرَتْ مُنْذُ أَرْبَعَةِ مِلْيَارَاتٍ مِنَ الْأَعْوَامِ، وَنَحْنُ بَنُو الْإِنْسَانِ لَمْ يَمْضِ عَلَى وُجُودِنَا سِوَى مِئَتَيْ أَلْفٍ مِنَ الْأَعْوَامِ، إِنَّهَا حِقْبَةٌ قَصِيرَةٌ لَكِنَّهَا كَانَتْ كَافِيَةً لِاخْتِلَالِ تَوَازُنِ أَرْضِنَا حَتَّى بَاتَتْ حَيَاتُنَا عَلَى كَوْكَبِنَا الْأَزْرَقِ مُهَدَّدَةً بِالزَّوَالِ. اِسْتَمِعُوا لِهَذِهِ الْحِكَايَةِ، حِكَايَتُنَا، حِكَايَةُ الْأَرْضِ.

شَاهِدُوا آثَارَ نَشْأَتِنَا الْأُولَى. فِي الْبِدَايَةِ لَمْ يَكُنْ كَوْكَبُنَا سِوَى كُتْلَةٍ نَارِيَّةٍ مُضْطَرِبَةٍ تَكَوَّنَتْ مِنَ الشَّمْسِ وَانْفَصَلَتْ عَنْهَا، سَحَابَةٍ مِنْ جُزَيْئَاتٍ غَازِيَّةٍ مُتَّصِلَةٍ تَسْبَحُ فِي الْفَضَاءِ شَأْنَهَا شَأْنُ الْعَدِيدِ مِنَ الْمَجْمُوعَاتِ الْمُمَاثِلَةِ فِي الْكَوْنِ، لَكِنْ هُنَا حَدَثَتِ الْمُعْجِزَةُ، مُعْجِزَةُ الْحَيَاةِ.

هَذِهِ الْحَيَاةُ الَّتِي نَعِيشُهَا الْيَوْمَ، حَيَاتُنَا لَيْسَتْ سِوَى حَلْقَةٍ مِنْ سِلْس